In [27]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


In [28]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np
import warnings
import scipy.misc
import random

warnings.filterwarnings("ignore")

In [37]:
pic_num_train = 40
pic_train = np.zeros((80,25*14,32*14))
label_train = np.zeros((80,2))
train_rnd_list = np.random.choice(80, 80, replace=False)
print(train_rnd_list)

for i in range (pic_num_train):
    pic_train[train_rnd_list[i],:,:] = scipy.misc.imread('Sparse_dataset_mean/train/negtive_pic/train_'+str(i)+ '.png')
    label_train[train_rnd_list[i],0] = 1

for i in range (pic_num_train):
    pic_train[train_rnd_list[i+pic_num_train],:,:] = scipy.misc.imread('Sparse_dataset_mean/train/positive_pic/train_'+str(i)+ '.png')
    label_train[[train_rnd_list[i+pic_num_train]],1] = 1

#plt.imshow(pic_train[79,:,:], cmap='gray')
#plt.show()

#print(label_train)

[ 8 37 19 55 24 47 63 34  2 52 16 72 36 22 32 78  9 27 43 15 40 21 25 51
 77 79 64 35  3  4 56 58 38 33 17 45  7 50 73 46 18 31 41 69  0 49 13 68
 10 53  1 20 26 28 12  6 76 66  5 11 23 29 14 44 70 65 62 75 39 30 57 74
 42 59 48 71 54 61 60 67]


In [38]:
pic_num_test = 500
pic_test = np.zeros((1000,25*14,32*14))
label_test = np.zeros((1000,2))
test_rnd_list = np.random.choice(1000, 1000, replace=False)


for i in range (pic_num_test):
    pic_test[test_rnd_list[i],:,:] = scipy.misc.imread('Sparse_dataset_mean/test/negtive_pic/test_'+str(i)+ '.png')
    label_test[test_rnd_list[i],0] = 1
    if ((i+1)%100 ==0):
        print(i + 1)

for i in range (pic_num_test):
    pic_test[test_rnd_list[i+pic_num_test],:,:] = scipy.misc.imread('Sparse_dataset_mean/test/positive_pic/test_'+str(i)+ '.png')
    label_test[test_rnd_list[i+pic_num_test],1] = 1
    if ((i+1)%100 ==0):
        print(i + 1 + pic_num_test)

100
200
300
400
500
600
700
800
900
1000


In [39]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Softmax,Activation,Dense
from keras.utils.np_utils import to_categorical
from keras.datasets import mnist
from sklearn.metrics import recall_score,f1_score,precision_score


In [40]:
pic_train=pic_train.reshape(-1,25*14,32*14,1)  #这里我们使用的是黑白图片
pic_test=pic_test.reshape(-1,25*14,32*14,1)
Y_train=to_categorical(label_train,num_classes= 2)
Y_test=to_categorical(label_test,num_classes= 2)


In [41]:
def VGG(X,Y):
    model=Sequential()
    #layer_1
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=X.shape[1:],padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',data_format='channels_last',kernel_initializer='uniform',activation='relu'))
    model.add(MaxPooling2D((2,2)))

    #layer_2
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(128,(2,2),strides=(1,1),padding='same',data_format='channels_last',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D((2,2)))

    #layer_3
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu'))
    model.add(Conv2D(256, (1, 1), strides=(1, 1), padding='same', data_format='channels_last', activation='relu'))
    model.add(MaxPooling2D((2,2)))
    #layer_4
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu'))
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', data_format='channels_last', activation='relu'))
    model.add(Conv2D(512, (1,1), strides=(1, 1), padding='same', data_format='channels_last', activation='relu'))
    model.add(MaxPooling2D((2,2)))

    #layer_5
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',data_format='channels_last',activation='relu'))
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', data_format='channels_last', activation='relu'))
    model.add(Conv2D(512, (1,1), strides=(1, 1), padding='same', data_format='channels_last', activation='relu'))
    model.add(MaxPooling2D((2,2)))

    model.add(Flatten())  #拉平
    model.add(Dense(4096,activation='relu'))
    model.add(Dense(4096,activation='relu'))
    model.add(Dense(1024,activation='relu'))
    #model.add(Dense(256,activation='relu'))
    model.add(Dense(2,activation='softmax'))

    model.summary()
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

    return model


In [ ]:
if __name__=="__main__":
    model=VGG(pic_train,label_train)
    model.fit(pic_train,label_train,batch_size=20,epochs=10)
    label_predict=model.predict(pic_train)
    print(label_predict)
    
    loss,acc=model.evaluate(pic_test,label_test)
    print(loss,acc)
    


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_66 (Conv2D)           (None, 350, 448, 64)      640       
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 350, 448, 64)      36928     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 175, 224, 64)      0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 175, 224, 128)     73856     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 175, 224, 128)     65664     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 87, 112, 128)      0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 87, 112, 256)     